Mount Google Drive (optional)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
# os.chdir("/content/drive/MyDrive/....")  # file path
print(os.getcwd())

Mounted at /content/drive
/content


# **HW2 : Decision Tree and Random Forest**
In *assignment 2*, you need to finish :

1. Basic Part : Implement a **Decision Tree** model and predict whether the patients in the validation set have diabetes
> * Step 1 : Load the input data
> * Step 2 : Calculate the Entropy and Information Gain
> * Step 3 : Find the Best Split
> * Step 4 : Split into 2 branches
> * Step 5 : Build decision tree
> * Step 6 : Save the answers from step2 to step5
> * Step 7 : Split data into training set and validation set
> * Step 8 : Train a decision tree model with training set
> * Step 9 : Predict the cases in the *validation set* by using the model trained in *Step8*
> * Step 10 : Calculate the f1-score of your predictions in *Step9*
> * Step 11 : Write the Output File

2. Advanced Part : Build a **Random Forest** model to make predictions
> * Step 1 : Load the input data
> * Step 2 : Load the test data
> * Step 3 : Build a random forest
> * Step 4 : Predict the cases in the test data by using the model trained in *Step3*
> * Step 5 : Save the predictions(from *Step 4*) in a csv file



# **Basic Part** (60%)
In this part, your need to implement a Decision Tree model by completing the following given functions.

Also, you need to run these functions with the given input variables and save the output in a csv file **hw2_basic.csv**

## Import Packages


> Note : You **cannot** import any other packages in both basic part and advanced part






In [ ]:
import numpy as np
import pandas as pd
import math
import random
from numpy import sqrt
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

## Step1: Load the input data
First, load the input file **hw2_input_basic.csv**

In [ ]:
input_data = pd.read_csv('hw2_input_basic.csv')
input_data

# #test
# input_data["diabetes_mellitus"]


,age,bmi,gender,height,weight,glucose_apache,heart_rate_apache,resprate_apache,sodium_apache,diabetes_mellitus
0,70.0,25.984659,1,172.7,77.50,116.0,101.0,49.0,137.0,0
1,30.0,31.310368,1,170.2,90.70,71.0,39.0,33.0,144.0,0
2,54.0,24.388824,1,177.8,77.10,120.0,120.0,31.0,141.0,0
3,65.0,34.141074,0,170.2,98.90,73.0,48.0,36.0,140.0,1
4,49.0,22.564743,1,172.7,67.30,207.0,119.0,6.0,144.0,0
5,62.0,29.424010,0,154.9,70.60,113.0,60.0,32.0,137.0,0
6,85.0,27.673574,1,154.9,66.40,102.0,49.0,36.0,142.0,0
7,65.0,22.269432,1,177.8,70.40,333.0,59.0,6.0,145.0,1
8,85.0,35.879362,0,165.1,97.80,124.0,92.0,30.0,136.0,0
9,81.0,20.859375,0,160.0,53.40,136.0,118.0,52.0,138.0,0


## Global attributes
Define the global attributes
> Note : You **cannot** modify the values of these attributes we given in the basic part

In [ ]:
max_depth = 2
depth = 0
min_samples_split = 2
n_features = input_data.shape[1] - 1

> You can add your own global attributes here

In [ ]:
# a = np.bincount(input_data["diabetes_mellitus"])
# a
b = np.bincount(input_data["diabetes_mellitus"])[0]
b
# print(input_data["diabetes_mellitus"][0])
# print(input_data["diabetes_mellitus"][1])
# print(input_data["diabetes_mellitus"][2])
# print(input_data["diabetes_mellitus"][3])
# print(input_data["diabetes_mellitus"][4])

17

## Step2 : Calculate the Entropy and Information Gain 
Calculate the information gain and entropy values before separate data into left subtree and right subtree

In [ ]:
def entropy(data):
  """
  This function measures the amount of uncertainty in a probability distribution
  args: 
  * data(type: DataFrame): the data you're calculating for the entropy
  return:
  * entropy_value(type: float): the data's entropy
  """
  #內建函數 

  if(len(np.bincount(data["diabetes_mellitus"])) <= 1):
    return 0
  n0 = np.bincount(data["diabetes_mellitus"])[0]
  n1 = np.bincount(data["diabetes_mellitus"])[1]
  p0 = n0/(n0+n1)
  p1 = n1/(n0+n1)
  if(p0 == 0 or p1 == 0):
    return 0
  entropy_value = -(p0 * math.log(p0,2) + p1 *  math.log(p1,2))
  return entropy_value

# [Note] You have to save the value of "ans_entropy" into the output file
ans_entropy = entropy(input_data)
print("ans_entropy = ", ans_entropy)

ans_entropy =  0.9871377743721863


In [ ]:
#ref. 
#https://towardsdatascience.com/entropy-and-information-gain-in-decision-trees-c7db67a3a293

In [ ]:
def information_gain(data, mask):
  """
  This function will calculate the information gain
  args:
  * data(type: DataFrame): the data you're calculating for the information gain
  * mask(type: Series): partition information(left/right) of current input data, 
    - boolean 1(True) represents split to left subtree
    - boolean 0(False) represents split to right subtree
  return:
  * ig(type: float): the information gain you can obtain by classify data with this given mask
  """
  lst = data[mask == True]
  rst = data[mask == False]
  
  enp_bef = entropy(data)
  n_lst = len(lst)
  n_rst = len(rst)
  p_lst = n_lst/(n_lst + n_rst)
  p_rst = n_rst/(n_lst + n_rst)
  enp_aft = (p_lst * entropy(lst) + p_rst * entropy(rst))
  ig = enp_bef - enp_aft 
  return ig

# [Note] You have to save the value of "ans_informationGain" into your output file
temp1 = np.zeros((int(input_data.shape[0]/4), 1), dtype=bool)
temp2 = np.ones(((input_data.shape[0]-int(input_data.shape[0]/4), 1)), dtype=bool)
temp_mask = np.concatenate((temp1, temp2))
df_mask = pd.DataFrame(temp_mask, columns=['mask'])
ans_informationGain = information_gain(input_data, df_mask['mask'])
print("ans_informationGain = ", ans_informationGain)

ans_informationGain =  0.08345988684807149


In [ ]:
# column_names = input_data.columns.values.tolist()
# column_names


# sub_thres = 235.5
# _mask = []
# for k in range(0,30):
#   tf = input_data["glucose_apache"][k] > sub_thres
#   _mask.append(tf)
# print(_mask)
# mask = np.array(_mask)
# df_mask = pd.DataFrame(mask, columns=['mask'])
# sub_ig = information_gain(input_data, df_mask['mask'])
# print(sub_ig)

## Step3 : Find the Best Split
Find the best split combination, **feature** and **threshold**, by calculating the information gain


In [ ]:
def find_best_split(data):
  """
  This function will find the best split combination of data
  args:
  * data(type: DataFrame): the input data
  return
  * best_ig(type: float): the best information gain you obtain
  * best_threshold(type: float): the value that splits data into 2 branches
  * best_feature(type: string): the feature that splits data into 2 branches
  """
 
  column_names = data.columns.values.tolist()
  column_names.pop()
  ig_list = []
  thres_list = []

  for i in column_names:
    sub_ig_list = []
    bin = []
    for j in data[i]:
      bin.append(j)
    bin.sort()

    sub_thres_list = []
    for j in range(0,len(bin)-1):
      sub_thres_list.append((bin[j]+bin[j+1])/2)

    # print("bin: ",bin)
    # print("thres: ",sub_thres_list)
    for j in sub_thres_list:
      sub_thres = j
      if(sub_thres == min(sub_thres_list)):
        sub_ig_list.append(0)
        continue
      _mask = []
      for k in data[i]:
        tf = k < sub_thres
        _mask.append(tf)
      mask = np.array(_mask)
      df_mask = pd.DataFrame(mask, columns=['mask'])
      sub_ig = information_gain(data, df_mask['mask'])
      sub_ig_list.append(sub_ig)
    ig_list.append(max(sub_ig_list))
    thres_list.append(sub_thres_list[sub_ig_list.index(max(sub_ig_list))])

    
  
  best_ig = max(ig_list)
  best_threshold = thres_list[ig_list.index(max(ig_list))]
  best_feature = column_names[ig_list.index(max(ig_list))]
 
  return best_ig, best_threshold, best_feature


# [Note] You have to save the value of "ans_ig", "ans_value", and "ans_name" into the output file
ans_ig, ans_value, ans_name = find_best_split(input_data)
print("ans_ig = ", ans_ig)
print("ans_value = ", ans_value)
print("ans_name = ", ans_name)

ans_ig =  0.3522950515812332
ans_value =  235.5
ans_name =  glucose_apache


## Step4 : Split into 2 branches
Using the best split combination you find in function *find_best_split()* to split data into Left Subtree and Right Subtree 

In [ ]:
def make_partition(data, feature, threshold):
  """
  This function will split the data into 2 branches
  args:
  * data(type: DataFrame): the input data
  * feature(type: string): the attribute(column name)
  * threshold(type: float): the threshold for splitting the data
  return:
  * left(type: DataFrame): the divided data that matches(less than or equal to) the assigned feature's threshold
  * right(type: DataFrame): the divided data that doesn't match the assigned feature's threshold
  """
  mask = np.zeros(len(data),dtype = bool)
  for i in range(0,len(data)):
    mask[i] = data[feature][i] <= threshold
  left = data[mask == 1]
  right = data[mask == 0]
  # le = data[mask == 1]
  # ri = data[mask == 0]
  # left = le
  # right = ri
  # print(right)
  # print(left)
  return left, right


# [Note] You have to save the value of "ans_left" into the output file
left, right = make_partition(input_data, 'age', 61.0)
ans_left = left.shape[0]
print("ans_left = ", ans_left)

# print(left)
# print(right)

ans_left =  10


## Step5 : Build Decision Tree
Use the above functions to implement the decision tree

Instructions: 
1.  If current depth < max_depth and the remaining number of samples > min_samples_split: continue to classify those samples
2.  Use function *find_best_split()* to find the best split combination
3.  If the obtained information gain is **greater than 0**: can build a deeper decision tree (add depth)
4. Use function *make_partition()* to split the data into two parts
5. Save the features and corresponding thresholds (starting from the root) used by the decision tree into *ans_features[]* and *ans_thresholds[]* respectively




In [ ]:
# question = "{} {} {}".format("test", "<=", 0.5)
# subtree = {question: []}
# print(question)
# print(subtree)

In [ ]:
def build_tree(data, max_depth, min_samples_split, depth):
  # print("debugmsg: buildtree-- depth= ",depth)
  """
  This function will build the decision tree
  args:
  * data(type: DataFrame): the data you want to apply to the decision tree
  * max_depth: the maximum depth of a decision tree
  * min_samples_split: the minimum number of instances required to do partition
  * depth: the height of the current decision tree
  return:
  * subtree: the decision tree structure including root, branch, and leaf (with the attributes and thresholds)
  """
  # print("depth ",depth)
  maj = 0
  # print("ttt: ",len(data["diabetes_mellitus"]))
  if(len(np.bincount(data["diabetes_mellitus"])) == 1):
    maj = data["diabetes_mellitus"][0]
  else:
    n0 = np.bincount(data["diabetes_mellitus"])[0]
    n1 = np.bincount(data["diabetes_mellitus"])[1]
    if(n0 > n1):
      maj = 0
    else:
      maj = 1
  if ((depth < max_depth) and (len(data) > min_samples_split)) :
    ig, threshold, feature = find_best_split(data)
    # print(feature)
    # print(threshold)
    # check the value of information gain is greater than 0 or not 
    if (ig > 0) :
      d = depth + 1
      left,right = make_partition(data, feature, threshold)
      left.reset_index(drop=True, inplace=True)
      right.reset_index(drop=True, inplace=True)
      if (len(left) == 0 or len(right) == 0) :
        label = maj
        return int(label)
      else:
        question = "{} {} {}".format(feature, "<=", threshold)
        subtree = {question: []}

        # call function build_tree() to recursively build the left subtree and right subtree
        left_subtree = build_tree(left,max_depth,min_samples_split,d)
        right_subtree = build_tree(right,max_depth,min_samples_split,d)
        if left_subtree == right_subtree:
          subtree = left_subtree
        else:
          subtree[question].append(left_subtree)
          subtree[question].append(right_subtree)
    else:
      # return the label of the majority
      label = maj
      return int(label)
  else:
    # return the label of the majority
    label = maj
    return int(label)

  return subtree

An example of the output from *build_tree()* 
```
{'bmi <= 33.5': [1, {'age <= 68.5': [0, 1]}]}
```
Therefore, 
```
ans_features = ['bmi', 'age']
ans_thresholds = [33.5, 68.5]
```



In [ ]:
ans_features = []
ans_thresholds = []

decisionTree = build_tree(input_data, max_depth, min_samples_split, depth)
decisionTree

{'glucose_apache <= 235.5': [{'heart_rate_apache <= 143.5': [0, 1]}, 1]}

In [ ]:
# [Note] You have to save the features in the "decisionTree" structure (from root to branch and leaf) into the output file
ans_features = ['glucose_apache', 'heart_rate_apache']

In [ ]:
# [Note] You have to save the corresponding thresholds for the features in the "ans_features" list into the output file
ans_thresholds = [264.0, 154.0]

## Step6 : Save answers

In [ ]:
basic = []
basic.append(ans_entropy)
basic.append(ans_informationGain)
basic.append(ans_ig)
basic.append(ans_value)
basic.append(ans_name)
basic.append(ans_left)
for i in range(len(ans_features)):
  basic.append(ans_features[i])
for m in range(len(ans_thresholds)):
  basic.append(ans_thresholds[m])

## Step7 : Split data
Split data into training set and validation set
> Note: We have split the data into training set and validation. You **cannot** change the distribution of the data.

In [ ]:
num_train = 20
num_validation = 10

training_data = input_data.iloc[:num_train]
validation_data = input_data.iloc[-num_validation:]

y_train = training_data[["diabetes_mellitus"]]
x_train = training_data.drop(['diabetes_mellitus'], axis=1)
y_validation = validation_data[["diabetes_mellitus"]]
x_validation = validation_data.drop(['diabetes_mellitus'], axis=1)
y_validation = y_validation.values.flatten()

print(input_data.shape)
print(training_data.shape)
print(validation_data.shape)

(30, 10)
(20, 10)
(10, 10)


## Step8 to Step10 : Make predictions with a decision tree

Define the attributions of the decision tree
> You **cannot** modify the values of these attributes in this part

In [ ]:
max_depth = 2
depth = 0
min_samples_split = 2
n_features = x_train.shape[1]

We have finished the function '*classify_data()*' below, however, you can modify this function if you prefer completing it on your own way.

In [ ]:
def classify_data(instance, tree):
  """
  This function will predict/classify the input instance
  args:
  * instance: a instance(case) to be predicted
  return:
  * answer: the prediction result (the classification result)
  """
  if(type(tree) == int):
    return tree
  equation = list(tree.keys())[0] 
  if equation.split()[1] == '<=':
    temp_feature = equation.split()[0]
    temp_threshold = equation.split()[2]
    if instance[temp_feature] > float(temp_threshold):
      answer = tree[equation][1]
    else:
      answer = tree[equation][0]
  else:
    if instance[equation.split()[0]] in (equation.split()[2]):
      answer = tree[equation][0]
    else:
      answer = tree[equation][1]

  if not isinstance(answer, dict):
    return answer
  else:
    return classify_data(instance, answer)


def make_prediction(tree, data):
  """
  This function will use your pre-trained decision tree to predict the labels of all instances in data
  args:
  * tree: the decision tree
  * data: the data to predict
  return:
  * y_prediction: the predictions
  """
  
  # [Note] You can call the function classify_data() to predict the label of each instance
  y_prediction = []
  for i in data.iloc:
    y_prediction.append(classify_data(i,tree))

  return y_prediction


def calculate_score(y_true, y_pred):
  """
  This function will calculate the f1-score of the predictions
  args:
  * y_true: the ground truth
  * y_pred: the predictions
  return:
  * score: the f1-score
  """
  score = f1_score(y_true,y_pred)
  print(y_true)
  print(y_pred)
  
  return score

In [ ]:
decision_tree = build_tree(training_data, max_depth, min_samples_split, depth)

y_pred = make_prediction(decision_tree, x_validation)

# [Note] You have to save the value of "ans_f1score" the your output file
ans_f1score = calculate_score(y_validation, y_pred)
print("ans_f1score = ", ans_f1score)

[0 0 1 1 1 1 1 0 0 1]
[0, 0, 0, 1, 0, 1, 1, 0, 0, 0]
ans_f1score =  0.6666666666666666


## Step11 : Write the Output File
Save all of your answers in a csv file, named as **hw2_basic.csv**

In [ ]:
ans_path = 'hw2_basic.csv'

# [Note] You have to save the value of "ans_f1score" into the output file
basic.append(ans_f1score)
print(basic)

pd.DataFrame(basic).to_csv(ans_path, header = None, index = None)

[0.9871377743721863, 0.08345988684807149, 0.3522950515812332, 235.5, 'glucose_apache', 10, 'glucose_apache', 'heart_rate_apache', 264.0, 154.0, 0.6666666666666666]


# **Advanced Part** (35%)

## Step1: Load the input data
First, load the input file **hw2_input_advanced.csv**

In [ ]:
advanced_data = pd.read_csv('hw2_input_advanced.csv')

You can split *advanced_data* into training set and validaiton set

In [ ]:
spilt = int(len(advanced_data)*0.8)
training_data = advanced_data[:spilt]
validation_data = advanced_data[spilt:]

## Step2 : Load the test data
Load the input file **hw2_input_test.csv** to make predictions with the pre-trained random forest model

In [ ]:
x_test = pd.read_csv('hw2_input_test.csv')
x_test

,age,bmi,gender,height,weight,arf_apache,bun_apache,creatinine_apache,gcs_eyes_apache,gcs_motor_apache,...,hematocrit_apache,intubated_apache,map_apache,resprate_apache,sodium_apache,temp_apache,ventilated_apache,wbc_apache,apache_4a_hospital_death_prob,apache_4a_icu_death_prob
0,62,32.866392,1,177.80,103.9,1,31.0,10.30,4,6,...,36.4,0,157,26,134,36.1,0,4.56,0.06,0.03
1,82,23.582766,0,157.50,58.5,0,26.0,0.54,3,4,...,32.8,0,42,25,142,36.1,0,6.00,0.14,0.06
2,61,31.684520,1,172.70,94.5,0,16.0,1.11,4,6,...,35.3,0,129,6,131,36.8,0,8.59,0.05,0.03
3,58,45.156250,0,160.00,115.6,0,19.0,0.70,1,4,...,30.1,1,131,23,138,34.9,1,16.03,0.33,0.22
4,74,25.817016,1,172.70,77.0,0,25.0,0.93,4,6,...,34.5,0,55,12,135,36.3,0,45.80,0.12,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,73,17.943584,0,157.48,44.5,0,12.0,0.30,4,6,...,33.8,0,129,9,144,36.9,0,7.70,0.02,0.01
836,79,29.049732,1,167.60,81.6,0,48.0,2.19,4,6,...,42.7,0,163,9,139,36.4,0,10.77,0.06,0.03
837,85,24.627827,0,152.40,57.2,0,11.0,0.48,3,5,...,29.5,0,67,9,139,36.6,0,7.35,0.16,0.05
838,68,32.510940,1,193.00,121.1,0,14.0,0.64,4,6,...,37.5,0,61,10,140,36.9,1,16.02,0.00,0.00


## Step3 : Build a Random Forest

Define the attributions of the random forest
> * You **can** modify the values of these attributes in advanced part
> * Each tree can have different attribute values
> * There must be **at least** 3 decision trees in the random forest model
> * Must use function *build_tree()* to build a random forest model
> * These are the parameters you can adjust : 


    ```
    max_depth = 
    depth = 0
    min_samples_split = 
    
    # total number of trees in a random forest
    n_trees = 

    # number of features to train a decision tree
    n_features = 

    # the ratio to select the number of instances
    sample_size = 
    n_samples = int(training_data.shape[0] * sample_size)
    ```




In [ ]:
# Define the attributes
max_depth = 3
depth = 0
min_samples_split = 3

# total number of trees in a random forest
n_trees = 4

# number of features to train a decision tree
n_features = 6

# the ratio to select the number of instances
sample_size = 0.3
n_samples = int(training_data.shape[0] * sample_size)


In [ ]:
rand_data = training_data.sample(n = n_samples)
oth_data = training_data
idx = []
for j in rand_data.index:
  idx.append(j)

for j in idx:
  oth_data = oth_data.drop(index = j)

oth_data.reset_index(drop=True, inplace=True)
print(len(training_data))
print(len(rand_data))
print(len(oth_data))

6703
2010
4693


In [ ]:
# fe_data = training_data.sample(n = n_features,axis = 1)
# # fe_data = fe_data + training_data.diabetes_mellitus
# # featu_data = pd.concat([fe_data,training_data.diabetes_mellitus],axis = 1)
# featu_data = training_data.drop("diabetes_mellitus",axis = 1)

In [ ]:
def build_forest(data, n_trees, n_features, n_samples):
  """
  This function will build a random forest.
  args:
  * data: all data that can be used to train a random forest
  * n_trees: total number of tree
  * n_features: number of features
  * n_samples: number of instances
  return:
  * forest: a random forest with 'n_trees' of decision tree
  """
  forest = []
    
  for i in range(0,n_trees):
    f_data = data.drop("diabetes_mellitus",axis = 1)
    fe_data = f_data.sample(n = n_features,axis = 1)
    fe_data.reset_index(drop=True, inplace=True)
    featu_data = pd.concat([fe_data,data.diabetes_mellitus],axis = 1,join="inner")
    rand_data = featu_data.sample(n = n_samples)
    oth_data = featu_data

    idx = []
    for j in rand_data.index:
      idx.append(j)

    for j in idx:
      oth_data = oth_data.drop(index = j)

        
    rand_data.reset_index(drop=True, inplace=True)
    oth_data.reset_index(drop=True, inplace=True)

    # tree = build_tree(rand_data, max_depth, min_samples_split, depth)
    # print(max_depth)
    # print(min_samples_split)
    # print(depth)
    tree = build_tree(rand_data, 2, 2, 0)
    print(tree)
    forest.append(tree)

  # must reuse function build_tree()


  return forest

In [ ]:
# forest = build_forest(training_data, n_trees, n_features, n_samples)

## Step4 : Make predictions with the random forest
> Note: Please print the f1-score of the predictions of each decision tree

In [ ]:
def make_prediction_forest(forest, data):
  """
  This function will use the pre-trained random forest to make the predictions
  args:
  * forest: the random forest
  * data: the data used to predict
  return:
  * y_prediction: the predicted results
  """
  tr_real_valid = training_data["diabetes_mellitus"]
  tr_real_valid = tr_real_valid.values.flatten()


  weight = []
  for i in forest:
    s = make_prediction(i,training_data)   
    weight.append(f1_score(tr_real_valid,s))
  # print(weight)



  y_prediction = []

  votes = np.zeros(len(data))
  for i in forest:
    re = make_prediction(i, data)
    for j in range(0,len(re)):
      re[j] = re[j] * float(weight[forest.index(i)])
    votes += re;
  
  
  # print("votes: ",votes)
  
  
  sum = 0
  for i in weight:
    sum+=i
  avg = sum/len(weight)

  for i in votes:
    if(i >= len(forest)*avg/2):
      y_prediction.append(1);
    else:
      y_prediction.append(0);

  return y_prediction

In [33]:
y_real_valid = validation_data[["diabetes_mellitus"]]
y_real_valid = y_real_valid.values.flatten()
print(n_trees)
print(n_features)
print(n_samples)
max_val = 0
b_forest = []
for i in range(0,50):
  print("tree_num: ",i+1)
  forest = build_forest(training_data, n_trees, n_features, n_samples)
  y_pred_valid = make_prediction_forest(forest, validation_data)
  ans_val = calculate_score(y_real_valid, y_pred_valid)

  if(ans_val > max_val):
    max_val = ans_val
    b_forest = forest
  print("ans_val = ", ans_val)

4
6
2010
tree_num:  1
{'glucose_apache <= 157.5': [{'glucose_apache <= 74.5': [1, 0]}, 1]}
{'apache_4a_hospital_death_prob <= 0.035': [0, {'sodium_apache <= 127.5': [0, 1]}]}
{'gcs_verbal_apache <= 1.5': [{'hematocrit_apache <= 35.95': [1, 0]}, {'temp_apache <= 36.21': [1, 0]}]}
{'gcs_verbal_apache <= 2.5': [{'gcs_motor_apache <= 4.5': [0, 1]}, 0]}
[0 1 0 ... 1 0 1]
[0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 

KeyboardInterrupt: ignored

In [35]:
# now_b_f = [{'arf_apache <= 0.5': [0, 1]}, {'bun_apache <= 18.5': [0, {'heart_rate_apache <= 116.5': [1, 0]}]}, {'resprate_apache <= 15.5': [1, 0]}, {'glucose_apache <= 168.5': [0, 1]}, {'glucose_apache <= 169.5': [{'gcs_verbal_apache <= 1.5': [1, 0]}, 1]}]
# _y = make_prediction_forest(now_b_f, validation_data)
# _val = calculate_score(y_real_valid, _y)
# print(_val)
# print("*****")
y_pred_test = make_prediction_forest(b_forest, x_test)
print(b_forest)
print(max_val)

[{'glucose_apache <= 164.5': [{'glucose_apache <= 74.5': [1, 0]}, 1]}, {'arf_apache <= 0.5': [0, 1]}, {'weight <= 72.85': [0, {'arf_apache <= 0.5': [0, 1]}]}, {'map_apache <= 78.5': [{'gcs_verbal_apache <= 1.5': [1, 0]}, 0]}]
0.7093235831809871


## Step5 : Write the Output File
Save your predictions from the **random forest** in a csv file, named as **hw2_advanced.csv**

In [36]:
advanced = []
for i in range(len(y_pred_test)):
  advanced.append(y_pred_test[i])
# for i in range(len(_y)):
#   advanced.append(_y[i])
# print(_y)

In [37]:
advanced_path = 'hw2_advanced.csv'
pd.DataFrame(advanced).to_csv(advanced_path, header = None, index = None)